In [32]:
import time 
import datetime
import pandas as pd

from datafuncs import process_cbpro_hist_data
from cbpro import PublicClient
from pandas.tseries.offsets import DateOffset

In [33]:
class CBProHistDataRetriever: 

    def __init__(self): 
        self.client = PublicClient()
        self.candles_per_request = 300
        
    def get_historical_prices(self, product: str, start_date: pd.Timestamp, end_date: pd.Timestamp, granularity: int): 
        offset = DateOffset(seconds = self.candles_per_request * granularity)
        date_range = pd.date_range(start=start_date, end=end_date, freq=offset) 
        date_range_iso = []
        for date in date_range: 
            date_range_iso.append(date.isoformat())

        historical_data = []
        for i in range(len(date_range_iso)-1): 
            start_date = date_range_iso[i]
            end_date = date_range_iso[i+1]
            data = self.client.get_product_historic_rates(
                product_id=product, 
                start=start_date, 
                end=end_date, 
                granularity=granularity
            )

            data.reverse() # The data comes in the wrong order.

            historical_data.extend(data)
            time.sleep(0.1)

        historical_data_df = process_cbpro_hist_data(historical_data)
        historical_data_df.drop_duplicates(inplace = True)
        return historical_data_df




In [34]:
data_helper = CBProHistDataRetriever()

In [37]:
product = 'BTC-USD'
granularity = 60
end = pd.Timestamp(datetime.datetime.now())
start_date = end - pd.Timedelta(days = 10)
candles = data_helper.get_historical_prices(
    product=product, start_date=start_date, end_date=end, granularity=granularity
)

In [42]:
end - pd.Timedelta(weeks = 52 * 3)

Timestamp('2019-08-01 11:59:08.498752')